# Simple moutain formation example

This notebook reproduces the [mountain example](https://fastscape-lem.github.io/fastscapelib-fortran/#_mountain_f90) provided in the fastscapelib-fortran library. This is the most common example of a landscape evolution model in action.

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import and inspect the model

We use the basic model available in [fastscape](https://fastscape.readthedocs.io/en/latest/).

In [ ]:
from fastscape.models import basic_model

In [ ]:
basic_model

In [ ]:
basic_model.visualize(show_inputs=True)

## Model setup

**Note**: One important difference between this model and the example shown in the fastscapelib-fortran library is that linear diffusion is here computed using the same topographic surface than the one used for computing stream power channel erosion. In fastscapelib-fortran, diffusion is always computed after applying channel erosion on the topographic surface, which makes it more "tolerant" to large time steps.

While it is possible to customize `basic_model` to mimic the behavior of fastscapelib-fortran (e.g., by replacing the diffusion process by another (sub)class), we keep the "apply-combine" approach here for erosion processes for more flexibility. Even if those processes are applied sequentially, setting large time steps has a significant impact on the solution as the latter is is only partially implicit (when considering the processes all together).

Here below we set time steps of 20000 years (5 times shorter than the same example in fastscapelib-fortran), for an overall simulation time of 4 million years.

In [ ]:
in_ds = xs.create_setup(
    model=basic_model,
    clocks={
        'time': np.arange(0, 4e6 + 2e4, 2e4),
        'out': np.arange(0, 4e6 + 1e5, 1e5),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [401, 401],
        'grid__length': [1e5, 1e5],
        'boundary__status': 'fixed_value',
        'uplift__rate': 1e-3,
        'spl': {
            'k_coef': 2e-6,
            'area_exp': 0.6,
            'slope_exp': 1.5
        },
        'diffusion__diffusivity': 1e-1
    },
    output_vars={
        'topography__elevation': 'out',
        'spl__chi': 'out',
    }
)

in_ds

## Run the model


In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=basic_model)

In [ ]:
out_ds

## Plot the outputs

Plot the $\chi$ integrated drainage area.

In [ ]:
import hvplot.xarray
from xshade import hillshade


chi_plot = out_ds.spl__chi.hvplot.image(
    x='x', y='y', clim=(0, 1e4),
    width=550, height=450,
    cmap='magma', groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap='gray', alpha=0.4,
    colorbar=False, hover=False, groupby='out'
)

chi_plot * hillshade_plot